In [1]:
# import basic modules
import sys
import os
import time
import numpy as np
from tqdm import tqdm
import gc
import torch
import argparse
import skimage.transform

# import custom modules
root_dir   = '/user_data/mmhender/imStat/'
sys.path.append(os.path.join(root_dir,'code'))
from model_src import fwrf_fit as fwrf_fit
from model_src import fwrf_predict as fwrf_predict
from model_src import texture_statistics_gabor, texture_statistics_pyramid, bdcn_features

bdcn_path = '/user_data/mmhender/toolboxes/BDCN/'

from model_fitting import initialize_fitting

fpX = np.float32



In [11]:
subject = 1
roi=None
up_to_sess = 5
shuffle_images = False
random_images = False
random_voxel_data = False

device = initialize_fitting.init_cuda()
nsd_root, stim_root, beta_root, mask_root = initialize_fitting.get_paths()

# decide what voxels to use  
voxel_mask, voxel_index, voxel_roi, voxel_ncsnr, brain_nii_shape = initialize_fitting.get_voxel_info(mask_root, beta_root, subject, roi)

# get all data and corresponding images, in two splits. always fixed set that gets left out
trn_stim_data, trn_voxel_data, val_stim_single_trial_data, val_voxel_single_trial_data, \
    n_voxels, n_trials_val, image_order = initialize_fitting.get_data_splits(nsd_root, beta_root, stim_root, subject, voxel_mask, up_to_sess, 
                                                                             shuffle_images=shuffle_images, random_images=random_images, random_voxel_data=random_voxel_data)

#device: 1
device#: 0
device name: GeForce RTX 2080 Ti

torch: 1.8.1+cu111
cuda:  11.1
cudnn: 8005
dtype: torch.float32
3794 voxels of overlap between kastner and prf definitions, using prf defs
unique values in retino labels:
[-1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20. 21. 22. 23. 24. 25.]
0 voxels of overlap between face and place definitions, using place defs
unique values in categ labels:
[-1.  0. 26. 27. 28. 30. 31. 32. 33.]
1535 voxels are defined (differently) in both retinotopic areas and category areas

14913 voxels are defined across all areas, and will be used for analysis

Loading numerical label/name mappings for all ROIs:
[1, 2, 3, 4, 5, 6, 7]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4', 'VO1', 'VO2', 'PHC1', 'PHC2', 'TO2', 'TO1', 'LO2', 'LO1', 'V3B', 'V3A', 'IPS0', 'IPS1', 'IPS2', 

In [17]:
trnsess3_stim = trn_stim_data[1358:1358+691,:,:,:]
trnsess3_vox = trn_voxel_data[1358:1358+691,:]

trnsess5_stim = trn_stim_data[1358+691+668:,:,:,:]
trnsess5_vox = trn_voxel_data[1358+691+668:,:]

idx = np.arange(10,20)
    
print(trnsess3_stim[idx,0,100,100])


print(trnsess3_vox[idx,100])


print(trnsess5_stim[idx,0,100,100])

print(trnsess5_vox[idx,100])


[0.69872946 0.17627059 0.86927843 0.39891842 0.66554666 0.06660549
 0.5294306  0.6332024  0.48060313 0.6773075 ]
[ 1.0194074  -0.7494248  -0.30403543 -0.6476215  -0.7589689   1.5729626
  1.6238645  -0.1131542  -1.977427   -0.723974  ]
[0.09777177 0.38532865 0.3680094  0.21786667 0.58716315 0.23282902
 0.41243845 0.5867522  0.43282825 0.5248392 ]
[-0.4275996 -0.6260706 -0.5709398 -1.3684993 -2.4325244 -0.7271438
  0.2321327 -1.1663527 -1.5724833 -0.9954472]


In [18]:
valsess3_stim = val_stim_single_trial_data[142:142+59,:,:,:]
valsess3_vox = val_voxel_single_trial_data[142:142+59,:]

valsess5_stim = val_stim_single_trial_data[142+59+82:,:,:,:]
valsess5_vox = val_voxel_single_trial_data[142+59+82:,:]

idx = np.arange(10,20)
    
print(valsess3_stim[idx,0,100,100])

print(valsess3_vox[idx,100])

print(valsess5_stim[idx,0,100,100])

print(valsess5_vox[idx,100])


[0.22517176 0.7677514  0.5758714  0.21600705 0.636611   0.46553883
 0.3349318  0.8771349  0.52526903 0.5758714 ]
[ 1.0512209e+00  1.9610877e+00 -1.5543070e+00 -1.8068455e-03
  8.8579065e-01  1.9865385e+00 -1.6433849e+00 -1.0039331e+00
 -6.2853342e-01 -8.1941462e-01]
[0.40976706 0.21146588 0.06514274 0.4048855  0.8566259  0.5108847
 0.6077961  0.37787688 0.25172627 0.30756944]
[ 1.2336764   0.17700192  0.3203421   1.2924826   0.08695484  0.36260903
  0.41957763 -0.96420646 -0.18134852 -0.7638977 ]


In [19]:
print(trn_stim_data[0:10,0,100,100])
print(trn_voxel_data[0:10,100])

print(val_stim_single_trial_data[0:10,0,100,100])
print(val_voxel_single_trial_data[0:10,100])

[0.6295412  0.08017726 0.5671694  0.7215106  0.09887294 0.6920259
 0.66238123 0.15907294 0.18663843 0.        ]
[-1.17288     0.78245544 -0.5046992  -1.4189155  -0.11104225 -0.95274293
 -1.2298566  -1.6986191  -0.5254179  -0.20168695]
[0.19799608 0.18083689 0.5912581  0.7458314  0.48001334 0.14401256
 0.7458314  0.29760784 0.6879035  0.7250235 ]
[-0.13953055 -0.01262802  2.3130558   0.0055009  -0.39333573 -0.48139057
  0.55972856  0.33959126 -0.06183509 -0.61606264]
